In [1]:
import os
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn import model_selection
from get_log_data import read_logs, get_digitized_logdata

X_train, X_validation, Y_train, Y_validation=get_digitized_logdata(2)


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
import time

import utils

# Define paramaters for the model

np.random.seed(1337)
learning_rate = 0.003
batch_size = 960
n_epochs = 630
n_train = 15000
n_test = 2000
train=X_train,Y_train
test=X_validation,Y_validation

In [3]:
X_train.shape,Y_train.shape,Y_validation.shape

((14853, 7), (14853, 2), (1887, 2))

In [4]:
# Step 2: Create datasets and iterator
train_data = tf.data.Dataset.from_tensor_slices(train)
# train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
img, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)	# initializer for train_data
test_init = iterator.make_initializer(test_data)	# initializer for train_data



In [5]:
# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# b is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = tf.matmul(X, w)
# shape of b depends on Y

# w = tf.get_variable(name='weights', shape=(X_train.shape[1], Y_train.shape[1]), initializer=tf.random_normal_initializer(0, 0.001))
# b = tf.get_variable(name='bias', shape=(1, Y_train.shape[1]), initializer=tf.zeros_initializer())
nhidden0 = 160
nhidden1 = 80
nhidden2 = 40
w0 = tf.get_variable(name='weights', shape=(X_train.shape[1], nhidden0), initializer=tf.random_normal_initializer(0, 0.001))
b0 = tf.get_variable(name='bias', shape=(1, nhidden0), initializer=tf.zeros_initializer())


w1 = tf.get_variable(name='weights1', shape=(nhidden0, nhidden1), initializer=tf.random_normal_initializer(0, 0.001))
b1 = tf.get_variable(name='bias1', shape=(1, nhidden1), initializer=tf.zeros_initializer())

w2 = tf.get_variable(name='weights2', shape=(nhidden1, nhidden2), initializer=tf.random_normal_initializer(0, 0.001))
b2 = tf.get_variable(name='bias2', shape=(1, nhidden2), initializer=tf.zeros_initializer())

wf = tf.get_variable(name='weightsf', shape=(nhidden2, Y_train.shape[1]), initializer=tf.random_normal_initializer(0, 0.001))
bf = tf.get_variable(name='biasf', shape=(1, Y_train.shape[1]), initializer=tf.zeros_initializer())
# #***************************************************

# W0 = tf.Variable(tf.random_normal([2, nhidden],stddev=0.1))
# b0 = tf.Variable(tf.zeros([nhidden]))

# W1 = tf.Variable(tf.random_normal([nhidden, 1],stddev=0.1))
# b1 = tf.Variable(tf.zeros([1]))

hidden1 = tf.matmul(img, w0) + b0

hidden2 = tf.matmul(hidden1, w1) + b1

hiddenf = tf.matmul(hidden2, w2) + b2
# yp = tf.matmul(tf.nn.relu(hidden), W1) + b1
# logits = tf.nn.softmax(yp,dim=0)
# #********************************************************

# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
logits = tf.matmul(tf.nn.relu(hiddenf), wf) + bf

# Step 5: define loss function
# use cross entropy of softmax of logits as the loss function


entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label, name='entropy')
loss = tf.reduce_mean(entropy, name='loss') # computes the mean over all the examples in the batch

# Step 6: define training op
# using gradient descent with learning rate of 0.01 to minimize loss
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
# optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)


# Step 7: calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
with tf.Session() as sess:
   
    start_time = time.time()
    sess.run(tf.global_variables_initializer())

    # train the model n_epochs times
    for i in range(n_epochs): 	
        sess.run(train_init)	# drawing samples from train_data
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
#         print()
        print('Average loss epoch {0}: {1} w1:{2}'.format(i, total_loss/n_batches,sess.run(w1)[0,0]))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test the model
    sess.run(test_init)			# drawing samples from test_data
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
    print(sess.run(w1))
    print('Accuracy {0}'.format(total_correct_preds/n_test))

Average loss epoch 0: 0.6940217725932598 w1:0.010669725947082043
Average loss epoch 1: 0.6703064385801554 w1:0.02415204606950283
Average loss epoch 2: 0.643114760518074 w1:0.023635871708393097
Average loss epoch 3: 0.6038780994713306 w1:0.026598386466503143
Average loss epoch 4: 0.6216896465048194 w1:0.024096744135022163
Average loss epoch 5: 0.6187548600137234 w1:0.023289533331990242
Average loss epoch 6: 0.6208184938877821 w1:0.019895009696483612
Average loss epoch 7: 0.613815113902092 w1:0.019222060218453407
Average loss epoch 8: 0.621694952249527 w1:0.018081046640872955
Average loss epoch 9: 0.6154240630567074 w1:0.01620233617722988
Average loss epoch 10: 0.6091997735202312 w1:0.016921106725931168
Average loss epoch 11: 0.5961907468736172 w1:0.015885602682828903
Average loss epoch 12: 0.6077510854229331 w1:0.0139440493658185
Average loss epoch 13: 0.5982673522084951 w1:0.012689372524619102
Average loss epoch 14: 0.6047499226406217 w1:0.010942279361188412
Average loss epoch 15: 0.59

Average loss epoch 125: 0.4635488926433027 w1:-0.08752734214067459
Average loss epoch 126: 0.458965826081112 w1:-0.09138533473014832
Average loss epoch 127: 0.4515968728810549 w1:-0.09106430411338806
Average loss epoch 128: 0.4476537087466568 w1:-0.09257745742797852
Average loss epoch 129: 0.4427587687969208 w1:-0.087983138859272
Average loss epoch 130: 0.44334696512669325 w1:-0.09211920201778412
Average loss epoch 131: 0.4396635047160089 w1:-0.08920591324567795
Average loss epoch 132: 0.4419886264950037 w1:-0.0920037105679512
Average loss epoch 133: 0.4413481834344566 w1:-0.08810856938362122
Average loss epoch 134: 0.44818978593684733 w1:-0.09028054028749466
Average loss epoch 135: 0.44754065200686455 w1:-0.0865744948387146
Average loss epoch 136: 0.45697357202880085 w1:-0.08694615215063095
Average loss epoch 137: 0.4501534625887871 w1:-0.08832346647977829
Average loss epoch 138: 0.4541026975493878 w1:-0.08738245069980621
Average loss epoch 139: 0.4438055814243853 w1:-0.08511231094598

Average loss epoch 247: 0.41070033772848547 w1:-0.014718783088028431
Average loss epoch 248: 0.39948245231062174 w1:-0.019413504749536514
Average loss epoch 249: 0.40228244266472757 w1:-0.01950002834200859
Average loss epoch 250: 0.39645337336696684 w1:-0.020373664796352386
Average loss epoch 251: 0.40251610497944057 w1:-0.019236866384744644
Average loss epoch 252: 0.39669902226887643 w1:-0.018048634752631187
Average loss epoch 253: 0.4110218440182507 w1:-0.01663597673177719
Average loss epoch 254: 0.41567975981161 w1:-0.015304545871913433
Average loss epoch 255: 0.4199371936265379 w1:-0.015993090346455574
Average loss epoch 256: 0.4234854537062347 w1:-0.012778625823557377
Average loss epoch 257: 0.44960095966234803 w1:-0.011182301677763462
Average loss epoch 258: 0.429100776091218 w1:-0.007237846963107586
Average loss epoch 259: 0.42455354472622275 w1:-0.010551616549491882
Average loss epoch 260: 0.4258428756147623 w1:-0.0021450205240398645
Average loss epoch 261: 0.4077325169928372 w

Average loss epoch 368: 0.3959283959120512 w1:0.03899442404508591
Average loss epoch 369: 0.39926377101801336 w1:0.03983676806092262
Average loss epoch 370: 0.388464636169374 w1:0.03845972195267677
Average loss epoch 371: 0.37978445598855615 w1:0.030081838369369507
Average loss epoch 372: 0.3896547141484916 w1:0.024735817685723305
Average loss epoch 373: 0.3944730560760945 w1:0.03502757102251053
Average loss epoch 374: 0.40576712926849723 w1:0.03731343522667885
Average loss epoch 375: 0.3997050621546805 w1:0.04167180135846138
Average loss epoch 376: 0.38568681944161654 w1:0.042351555079221725
Average loss epoch 377: 0.3772180525120348 w1:0.03731011599302292
Average loss epoch 378: 0.38021267275325954 w1:0.03645487129688263
Average loss epoch 379: 0.3743957120459527 w1:0.0368412546813488
Average loss epoch 380: 0.3800802263431251 w1:0.03417016938328743
Average loss epoch 381: 0.3775295470841229 w1:0.03621817007660866
Average loss epoch 382: 0.3859411512967199 w1:0.03906724229454994
Aver

Average loss epoch 490: 0.3673998904414475 w1:0.04277825355529785
Average loss epoch 491: 0.37291251542046666 w1:0.036991775035858154
Average loss epoch 492: 0.3826207430101931 w1:0.04998079314827919
Average loss epoch 493: 0.3829288687556982 w1:0.050966039299964905
Average loss epoch 494: 0.3800260794814676 w1:0.039147622883319855
Average loss epoch 495: 0.37174551328644156 w1:0.05225830525159836
Average loss epoch 496: 0.41197336092591286 w1:0.05030740424990654
Average loss epoch 497: 0.3973605721257627 w1:0.04389108344912529
Average loss epoch 498: 0.3954548202455044 w1:0.044894300401210785
Average loss epoch 499: 0.42324177036061883 w1:0.03413090109825134
Average loss epoch 500: 0.41909094620496035 w1:0.032173652201890945
Average loss epoch 501: 0.421998496633023 w1:0.03652212396264076
Average loss epoch 502: 0.38857441931031644 w1:0.04479243978857994
Average loss epoch 503: 0.3713708915747702 w1:0.04607373848557472
Average loss epoch 504: 0.3708572145551443 w1:0.04565399885177612


Average loss epoch 614: 0.3607076173648238 w1:0.04286846145987511
Average loss epoch 615: 0.34952586772851646 w1:0.046872809529304504
Average loss epoch 616: 0.3665946847759187 w1:0.04701468348503113
Average loss epoch 617: 0.3547163854818791 w1:0.04532971978187561
Average loss epoch 618: 0.3836285504512489 w1:0.048119522631168365
Average loss epoch 619: 0.36339387425687164 w1:0.0465269573032856
Average loss epoch 620: 0.34901002468541265 w1:0.0539366751909256
Average loss epoch 621: 0.3809899149928242 w1:0.05118333548307419
Average loss epoch 622: 0.36484773689880967 w1:0.05039519444108009
Average loss epoch 623: 0.38298981124535203 w1:0.04989571124315262
Average loss epoch 624: 0.39269666001200676 w1:0.03455658257007599
Average loss epoch 625: 0.388624909799546 w1:0.05199047550559044
Average loss epoch 626: 0.43680308759212494 w1:0.053145650774240494
Average loss epoch 627: 0.3846966288983822 w1:0.04352029040455818
Average loss epoch 628: 0.3609949378296733 w1:0.04218257591128349
Ave